# Diffie Hellman

- Establish a shared secret between two parties over an insecure channel.
- Provides confidentiality foundation for symmetric encryption; does not provide authentication/confidentiality by itself.

The key is to use public encryption to solve the key distribution 

- Distribution of public keys via certificates with a Third Trusted Party
    - Can be donde using:
        - Public announcement 
        - Public available directory
        - Public Key Authority
        - Centralized way? no bottleneck distributed
- Use of public jet encryption to distribute secret keys



### Primitive root (notes)

- Definition  
    A number $g$ is a primitive root modulo $n$ if the powers of $g$ generate all units modulo $n$ (the multiplicative group $(\mathbb{Z}/n\mathbb{Z})^*$). Equivalently, the multiplicative order of $g$ modulo $n$ equals $\phi(n)$ (Euler's totient).

- Important special case (primes)  
    For a prime $p$ the group $(\mathbb{Z}/p\mathbb{Z})^*$ is cyclic of order $p-1$, so a primitive root modulo $p$ is an element of order $p-1$. Primitive roots always exist for prime moduli.

- Practical test for primes  
    Let $p$ be prime and let the distinct prime factors of $p-1$ be $q_1, q_2, \dots, q_k$. A candidate $g$ ($2 \le g < p$) is a primitive root mod $p$ iff for every $i$:

    $g^{(p-1)/q_i} \not\equiv 1 \pmod p.$

    If any test equals $1$, $g$ is not a generator.

- How to find one (algorithm sketch)
    1. Factor $p-1$ to get its prime factors $\{q_i\}$.  
    2. For $g = 2, 3, 4, \dots$ check $g^{(p-1)/q_i} \pmod p$ for all $q_i$.  
    3. The first $g$ that passes all checks is a primitive root.

- Small example (p = 7)  
    $p-1 = 6$, prime factors $\{2, 3\}$. Test $g = 3$:  

    $3^{6/2} = 3^3 \equiv 6 \pmod 7$
    $3^{6/3} = 3^2 \equiv 2 \pmod 7$  

    So $3$ is a primitive root mod $7$. Powers: $3^1\dots 3^6 \equiv 3,2,6,4,5,1$ (all nonzero residues).

- Relevance to Diffie–Hellman  
    In Diffie–Hellman you pick a prime $p$ and a generator $g$ (a primitive root modulo $p$ or at least an element of large order) so that repeated powers of $g$ produce the key space used for the exchange.

# Finding other primitive roots

If $a$ is a primitive root modulo a prime $p$, then for any integer $r$ the element $a^r$ is a primitive root modulo $p$ if and only if $\gcd(r, p-1) = 1$.

Why: the order of $a$ modulo $p$ is $p-1$. The order of $a^r$ is $\frac{p-1}{\gcd(r,p-1)}$. So $a^r$ has order $p-1$ (i.e. is a primitive root) exactly when $\gcd(r,p-1)=1$.

Consequence: all primitive roots modulo $p$ are obtained as $a^r$ where $r$ ranges over the integers coprime to $p-1$. The number of primitive roots modulo $p$ is $\phi(p-1)$ (Euler's totient of $p-1$).

Example (p = 59, a = 2):

- Here $p-1 = 58 = 2 dot 29$. The integers coprime to 58 are those not divisible by 2 or 29. There are $\phi(58)=28$ such values, so there are 28 primitive roots modulo 59.

- To build other primitive roots from $2$, compute $2^r \mod 59$ for each $r$ with $1 e r < 58$ and $\gcd(r,58)=1$. A few examples:
    - $r=3$: $2^3 = 8$ so $8$ is a primitive root mod 59.
    - $r=5$: $2^5 = 32$ so $32$ is a primitive root mod 59.
    - $r=7$: $2^7 = 128 quiv 10 mod{59}$ so $10$ is a primitive root mod 59.

Practical algorithm to list primitive roots given one primitive root $a$:
1. Factor $p-1$ and compute the list $R = r : 1 e r < p-1,\ cd(r,p-1)=1$.
2. For each $r$ in $R$ compute $a^r \mod p$; the results (distinct) are exactly the primitive roots modulo $p$.

Note: when implementing this, avoid duplicates (different $r$ can give the same residue); using a set to collect results handles that automatically.



## Diffi Hellman Key Exchange Algorithm

The goal is to enable two uses to securely exchange a key 

Each user generate their key 
- chooses a secret key numbre 
- compute their public key ya = a^xa mod p
- compute their public key yb = a^xb mod p
- To get the session key for users 
- KAb = a^xaxb mod p
= ya^xb mod  p  b compute
= yb^xa mod  p  b compute

### example

alice and bob  agree  p=353 and a =3
alice selects random xa =97 
alice computes  